# Tutotial 02 - Interacting With and Plotting Ehys Data 

In Tutorial 1 - Introduction to pyeCAP, we covered loading and making a simple plot of a single data set. Here we will expand upon this to cover  working with channel types and channel names as well as loading multiple data sets. We can start by importing pyeCAP and loading a data set.

## Working with Channels and Channel Types

In this section, we'll explore how to work with channels and channel types in our dataset. This dataset was collected with a Tucker Davis Technologies (TDT) recording system. The Ephys class automatically maps the data on your computer's hard disk. This dataset contains two streams of raw (unfiltered) Ephys data, called 'RawE' and 'RawG' that we collected in different stores within the TDT data tank. 

For more information on how TDT data types are stored and accessed, you can refer to the [TDT Python SDK documentation](https://www.tdt.com/support/python-sdk/).

In the previous tutorial, we loaded all data stores detected in the TDT data tank. In this tutorial, we'll specify only the two raw data stores. 

Let's start by importing the necessary libraries and loading our dataset.

In [ ]:
import pyeCAP

# path to directory containing the TDT tank
directory = r"pnpig191126-191204-174838"
data = pyeCAP.Ephys(directory, stores=['RawE', 'RawG'])

Now that we've loaded our data, let's examine the number of channels in our dataset. We can do this using the `ch_names` property of an Ephys object.

In [ ]:
# This will return a list of channel names in our dataset.
data.ch_names

In this experiment, 'RawE' contains data recorded from 4 LIFE electrodes inserted into the swine vagus nerve, and 'RawG' contains data recorded from EMG electrodes inserted into relevent muscle groups. Although 'RawE' contains 4 channels of data, only 3 electrodes in channels 'RawG 1'-'RawG 3' were used in the experiment. We can start by removing the extra channel of EMG data from the data set and renaming the channels to something that makes more sense. We will also set the channel types to either 'EMG' or 'LIFE' so that we can independently interact with these channels through the rest of our analysis.

It is important to note that all operations that change an object within pyeCAP return a new object. For those familiar with other scientific conputing languages in python like numpy or pandas, pyeCAP objects are never modified in place like numpy arrays or padas dataframes can be.

In [ ]:
# Starting by removing the excess channel 'RawG 4'. We could do this either by the index in the channel list (7) or using the channel name directly. 
used_data = data.remove_ch('RawG 4')

# We can rename the channels using the set_ch_names method and providing a list of new channel names that matches the number of channels in the data set.
renamed_data = used_data.set_ch_names(['LIFE 1', 'LIFE 2', 'LIFE 3', 'LIFE 4', 'EMG 1', 'EMG 2', 'EMG 3'])

# Lets also set the types of the individual channels so that we can indepentently interact with LIFE and EMG channels
renamed_data = renamed_data.set_ch_types(['LIFE', 'LIFE', 'LIFE', 'LIFE', 'EMG', 'EMG', 'EMG'])

In [ ]:
data.shape

We can compare a plot of data with a plot of renamed_data to see the changes in the number of channels and in the prescribed channel names. We can plot these side by side in figure subplots using matplotlib. 

In [ ]:
# import matplotlib
import matplotlib.pyplot as plt

# Set up figure axes that we can plot on
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6))
fig.suptitle('Organization and renaming of channels is automatically reflected in plots of the data')

# Label axes and plot data. 
ax1.set_title('data')
# The axis attribute is used to provide a matplotlib axis to plot on.
# Show is set to False to prevent the plot function from automatically showing the plot within the jupyter notebook.
data.plot(axis = ax1, x_lim=(0,5), show=False)
ax2.set_title('renamed_data')
renamed_data.plot(axis = ax2, x_lim=(0,5), show=False)

# Tell matplotlib to show the plot within a jupyter notebook
plt.show()

## Interactive Plotting of Data

One of the main goals of pyeCAP is to allow for interactively working with electrophysiology, and in particular compound action potential, data during or shortly after an experiment. As such, many plot types have been designed as jupyter notebook widgets and can be taken advantage of to interactively work with data or to view data in real time.

The most simple example of this is the plot method for the ephys class. By passhing show='notebook' as an attribute to the plot method, an interactive plot is shown within the current jupyter notebook output cell. 

However, before using interactive plots we need to change matplotlib to use an interactive backend. We can do this using the a Jupyter Notebook magic (for more information on jupyter notebook magics see this tutorial: https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-matplotlib). In our case we want to use the magic command '%matplotlib widget'. If we switch back to '%matplotlib inline' which is the default mode for matplotlib, it will break our connection to any widgets in use. As such it is a good idea to only work with matplotlib in inline or widget mode within a given notebook.

**If you encounter issues with interactive plots and are running pyeCAP in an anaconda environment it is recommended that you reinstall the ipyml package used for interactive plotting by running 'conda install -c conda-forge ipympl'.**

In [ ]:
%matplotlib widget
renamed_data.plot(x_lim = (0,0.1), show='notebook')

Any plot that we create can be used with the matplotlib widget backend. As an example, we can replicate the side-by-side plot of data and renamed data that we created above using the widget backend instead of the inline backend.

In [ ]:
import matplotlib.pyplot as plt

# Set up figure axes that we can plot on
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))
fig.suptitle('Organization and renaming of channels is automatically reflected in plots of the data')

# Label axes and plot data. 
ax1.set_title('data')
# The axis attribute is used to provide a matplotlib axis to plot on.
# Show is set to False to prevent the plot function from automatically showing the plot within the jupyter notebook.
data.plot(axis = ax1, x_lim=(0,5), show=False)
ax2.set_title('renamed_data')
renamed_data.plot(axis = ax2, x_lim=(0,5), show=False)

# Tell matplotlib to show the plot within a jupyter notebook
plt.show()

## Directly Working with Ephys Data as an Array

It is common that we will want extract portions of our data to directly work with as an array of values. In python the normal way to work with array data is as a numpy array. An introduction to numpy arrays can be found here (https://numpy.org/doc/stable/user/quickstart.html). For all intents and purposes, we can treat an ephys object as numpy arrays with shape equal to *number of channels* x *number of data points*.

In [ ]:
renamed_data.shape

Data can be directly accessed by indexing into the data set. For instance we can return the first 10 elements of channel 0 in *renamed_data* using the follwoing code. All numpy indexing methods are supported.

In [ ]:
renamed_data[0,0:10]

In addition to working with Ephys data as a numpy array, there are a number of useful properties that are worth knowing about. Some of the most useful ones are:

* sample_rate
 - Gives the sampling rate for the data set.
* time
 - Returns as array starting at zero of time points corresponding to each data point within the data set.
* ch_names
 - Returns a list of names for each channel within the data set. 
* ch_types
 - Returns a list of types assigned to each channel in the data set.
 
 
 For instance, we can easily see that the sampling rate of *renamed_data* is a little under 25kHz:

In [ ]:
renamed_data.sample_rate

We can also check that the channel names match what we previously set them to:

In [ ]:
renamed_data.ch_names

Lastly we can see that there are two channel types contained within *renamed_data*. These channel types match what we previously assigned to the data set:

In [ ]:
renamed_data.ch_types